In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler

# 设置中文显示
plt.rcParams['font.sans-serif'] = ['Microsoft YaHei', 'SimHei']
plt.rcParams['axes.unicode_minus'] = False

# 1. 加载数据
df = pd.read_csv('../../data/intermediate/初步清洗_比赛数据_2.csv')

# 2. 定义资源相关的字段（已经归一化，0-1范围）
resource_columns = [
    '教室设备满意度', '实训室满意度', '图书馆满意度',
    '网络资源满意度', '体育设施满意度', '住宿条件满意度'
]

# 3. 数据预处理：确保资源字段都是数值型
for col in resource_columns:
    df[col] = pd.to_numeric(df[col], errors='coerce')

# 处理缺失值：用列均值填充
df[resource_columns] = df[resource_columns].fillna(df[resource_columns].mean())

# 方法2：基于相关性的权重（参考EHI计算方式）
# 这里使用与"学校整体满意度"的相关性作为权重依据
correlation_weights = {}
target_variable = '学校整体满意度'

for col in resource_columns:
    # 计算每项资源满意度与整体满意度的相关性
    corr = df[col].corr(df[target_variable])
    correlation_weights[col] = abs(corr)  # 取绝对值

# 归一化权重
total_weight = sum(correlation_weights.values())
if total_weight > 0:
    correlation_weights = {k: v/total_weight for k, v in correlation_weights.items()}
else:
    # 如果所有相关性都为0，则平均分配权重
    correlation_weights = {k: 1/len(resource_columns) for k in resource_columns}

print("基于相关性的资源权重分配:")
for resource, weight in correlation_weights.items():
    print(f"  {resource}: {weight:.3f}")

# 使用相关性权重计算RPI
df['RPI_weighted'] = 0
for col in resource_columns:
    df['RPI_weighted'] += df[col] * correlation_weights[col] * 100

# 限制分数在0-100范围内
df['RPI_weighted'] = np.clip(df['RPI_weighted'], 0, 100)

# 计算并显示RPI_weighted指标的标准差
rpi_std = df['RPI_weighted'].std()
rpi_mean = df['RPI_weighted'].mean()
rpi_min = df['RPI_weighted'].min()
rpi_max = df['RPI_weighted'].max()

print("\n=== RPI_weighted指标统计信息 ===")
print(f"平均值: {rpi_mean:.4f}")
print(f"标准差: {rpi_std:.4f}")
print(f"最小值: {rpi_min:.4f}")
print(f"最大值: {rpi_max:.4f}")
print(f"变异系数: {rpi_std/rpi_mean:.4f}")
print(f"数据范围: [{rpi_min:.4f}, {rpi_max:.4f}]")

# 显示分布情况
print(f"\n=== RPI_weighted分布情况 ===")
print(df['RPI_weighted'].describe())

# 绘制RPI_weighted的分布直方图
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.hist(df['RPI_weighted'], bins=30, alpha=0.7, color='skyblue', edgecolor='black')
plt.axvline(rpi_mean, color='red', linestyle='--', label=f'平均值: {rpi_mean:.2f}')
plt.axvline(rpi_mean + rpi_std, color='orange', linestyle=':', label=f'平均值±标准差')
plt.axvline(rpi_mean - rpi_std, color='orange', linestyle=':')
plt.xlabel('RPI_weighted得分')
plt.ylabel('频数')
plt.title('RPI_weighted分布直方图')
plt.legend()
plt.grid(alpha=0.3)

plt.subplot(1, 2, 2)
# 箱线图显示分布
plt.boxplot(df['RPI_weighted'].dropna(), vert=False)
plt.xlabel('RPI_weighted得分')
plt.title('RPI_weighted箱线图')
plt.grid(alpha=0.3)

plt.tight_layout()
plt.show()

# 5. 按专业分组计算平均RPI（用于热力图）
# 选择加权RPI作为主要指标
major_rpi = df.groupby(['学院', '专业']).agg({
    'RPI_weighted': 'mean',
    '教室设备满意度': 'mean',
    '实训室满意度': 'mean',
    '图书馆满意度': 'mean',
    '网络资源满意度': 'mean',
    '体育设施满意度': 'mean',
    '住宿条件满意度': 'mean'
}).round(3)

# 显示专业级别的标准差
print(f"\n=== 专业级别RPI统计 ===")
print(f"专业数量: {len(major_rpi)}")
print(f"专业间RPI平均值: {major_rpi['RPI_weighted'].mean():.4f}")
print(f"专业间RPI标准差: {major_rpi['RPI_weighted'].std():.4f}")
print(f"专业间RPI最小值: {major_rpi['RPI_weighted'].min():.4f}")
print(f"专业间RPI最大值: {major_rpi['RPI_weighted'].max():.4f}")

# 6. 创建热力图数据
heatmap_data = major_rpi[resource_columns]

# 标准化数据（Z-score）以便更好地显示差异
scaler = StandardScaler()
heatmap_data_standardized = pd.DataFrame(
    scaler.fit_transform(heatmap_data),
    index=heatmap_data.index,
    columns=heatmap_data.columns
)

# 7. 绘制热力图
plt.figure(figsize=(14, 10))

# 创建热力图
sns.heatmap(
    heatmap_data_standardized,
    cmap='RdYlGn',  # 红-黄-绿色彩映射，红色表示低分，绿色表示高分
    center=0,       # 中心点为0（Z-score的均值）
    annot=True,     # 显示数值
    fmt='.2f',      # 数值格式
    linewidths=0.5,
    cbar_kws={'label': '标准化满意度得分 (Z-score)'}
)

plt.title('各专业资源满意度热力图\n(按Z-score标准化)', fontsize=16, fontweight='bold', pad=20)
plt.xlabel('资源类型', fontsize=12)
plt.ylabel('专业', fontsize=12)
plt.xticks(rotation=45, ha='right')
plt.tight_layout()

# 8. 保存结果
# 保存专业级别的RPI数据
major_rpi.to_csv('../../data/processed/专业资源感知度指标.csv', encoding='utf-8-sig')

# 保存学院级别的RPI数据
college_rpi = df.groupby('学院').agg({
    'RPI_weighted': ['mean', 'std', 'count'],
    '教室设备满意度': 'mean',
    '实训室满意度': 'mean',
    '图书馆满意度': 'mean',
    '网络资源满意度': 'mean',
    '体育设施满意度': 'mean',
    '住宿条件满意度': 'mean'
}).round(3)
college_rpi.to_csv('../../data/processed/学院资源感知度指标.csv', encoding='utf-8-sig')

print("\n资源感知度分析完成！")
print(f"总体RPI平均分: {df['RPI_weighted'].mean():.2f}")
print(f"总体RPI标准差: {df['RPI_weighted'].std():.2f}")
print(f"最高专业RPI: {major_rpi['RPI_weighted'].max():.2f}")
print(f"最低专业RPI: {major_rpi['RPI_weighted'].min():.2f}")

plt.show()

# 额外分析：各资源项的统计信息
print(f"\n=== 各资源项统计信息 ===")
for col in resource_columns:
    col_mean = df[col].mean()
    col_std = df[col].std()
    print(f"{col}: 平均值={col_mean:.4f}, 标准差={col_std:.4f}")